In [6]:
import praw
import pandas as pd
from datetime import datetime
from tqdm import tqdm
import time
import prawcore

# --- Reddit API credentials ---
reddit = praw.Reddit(
    client_id="KM3SXXKIOVv0T2Q7YIDO-g",
    client_secret="P5eVSfGwsouYLMYfXEe1GoMSJLj4jA",
    user_agent="healthcare-analysis by u/slmdatascience"
)

# --- Parameters ---
subreddits = ["Medicare", "Medicaid", "HealthInsurance"]
keywords = ["medicare", "medicaid"]
max_posts = 1000         # total posts per subreddit
comments_per_post = 3    # top comments per post
sleep_between_posts = 2  # seconds

# --- Storage ---
posts = []
comments = []

# --- Helper function to fetch top comments safely ---
def fetch_top_comments(submission, top_n):
    try:
        submission.comments.replace_more(limit=0)
        return submission.comments[:top_n]
    except prawcore.exceptions.TooManyRequests:
        print("Rate limit hit while fetching comments. Sleeping 60 seconds...")
        time.sleep(60)
        submission.comments.replace_more(limit=0)
        return submission.comments[:top_n]
    except Exception as e:
        print(f"Error fetching comments for post {submission.id}: {e}")
        return []

# --- Scrape each subreddit ---
for sub in subreddits:
    subreddit = reddit.subreddit(sub)
    collected = 0
    last_timestamp = None

    print(f"\n🔍 Collecting up to {max_posts} posts from r/{sub}...")

    while collected < max_posts:
        params = {}
        if last_timestamp:
            params["before"] = int(last_timestamp)

        try:
            batch = list(subreddit.new(limit=1000, params=params))
        except prawcore.exceptions.TooManyRequests:
            print("Rate limit hit while fetching posts. Sleeping 60 seconds...")
            time.sleep(60)
            continue
        except Exception as e:
            print(f"Error fetching posts: {e}")
            break

        if not batch:
            print("No more posts available in this subreddit.")
            break

        for submission in tqdm(batch):
            last_timestamp = submission.created_utc

            text = (submission.title + " " + submission.selftext).lower()
            if not any(k in text for k in keywords):
                continue

            posts.append({
                "id": submission.id,
                "title": submission.title,
                "selftext": submission.selftext,
                "subreddit": sub,
                "score": submission.score,
                "num_comments": submission.num_comments,
                "created_utc": submission.created_utc,
                "date": datetime.fromtimestamp(submission.created_utc),
                "url": submission.url
            })

            # Top comments
            top_comments = fetch_top_comments(submission, comments_per_post)
            for c in top_comments:
                comments.append({
                    "post_id": submission.id,
                    "comment_id": c.id,
                    "body": c.body,
                    "score": c.score,
                    "created_utc": c.created_utc,
                    "date": datetime.fromtimestamp(c.created_utc)
                })

            collected += 1
            if collected >= max_posts:
                break

            time.sleep(sleep_between_posts)

# --- Create DataFrames and save ---
df_posts = pd.DataFrame(posts).drop_duplicates(subset=["id"])
df_comments = pd.DataFrame(comments).drop_duplicates(subset=["comment_id"])

df_posts.to_csv("reddit_posts_2.csv", index=False)
df_comments.to_csv("reddit_comments_2.csv", index=False)

print(f"\n✅ Done! Collected {len(df_posts)} posts and {len(df_comments)} comments across {len(subreddits)} subreddits.")



🔍 Collecting up to 1000 posts from r/Medicare...


 36%|██████████████▊                          | 361/997 [09:22<16:30,  1.56s/it]



🔍 Collecting up to 1000 posts from r/Medicaid...


 19%|███████▋                                 | 184/988 [05:38<24:40,  1.84s/it]



🔍 Collecting up to 1000 posts from r/HealthInsurance...


 20%|████████▏                                | 198/999 [01:00<04:03,  3.28it/s]



✅ Done! Collected 1698 posts and 4151 comments across 3 subreddits.
